# 📊 **QA Script for Population Projections**

## 📝 **Introduction**
This notebook aims to identify outliers in the GLA population projection data. The analysis involves loading the dataset, preprocessing the data, defining utility functions, performing outlier detection, and presenting the results through visualisations.

### 🎯 **Goals**
The analysis will focus on the following objectives:
- **Load and preprocess** the population projections dataset.
- Define utility functions.
- Perform **basic checks** on the dataset:
  - Range of years covered.
  - Missing values.
  - Duplicates.
  - Descriptive statistics.
  - Breakdown by components.
  - Detecting negative values.
  - Age group ranges.
- **Outlier Detection** over time for each component:
  - Identify outliers using **Z-scores** and **Robust Z-scores**.
  - Analyse by **component**, **ward**, and **borough**.
  - Handle **infinite values** separately.
- **Total Population Outliers**:
  - Use Z-scores and Robust Z-scores for comparison.
  - Perform **cross-sectional comparisons**: Examine changes between boroughs and wards for a given year.
  - Conduct **temporal comparisons**: Measure percentage changes between years for both boroughs and wards.
  - Handle **infinite values** separately.
- **Gender Outliers**:
  - Investigate abnormal **gender ratios**.
  - Analyse by component.
  - Adjust the **outlier standard deviation thresholds** as needed based on different components.
- **Key Visualisations**:
  - Display the distribution of components.
  - Group data by **age ranges**.
  - Visualise **yearly totals**.
  - Show yearly total trends over time, broken down by components.
  - Create **population pyramids**.
- **Collate Outliers**: Summarise and determine the key outlier rows.

---

## 📂 **Dataset**
How should the dataset be structured?

The functions are designed to take datasets in the form produced by the GLA Population Projection Workflow. In the current iteration of this workflow, the dataset contains several key columns and components as outlined below:

### Main Columns
- **gss_code**: Borough geocode (e.g., GSS code).
- **la_name**: Local Authority name.
- **Year**: The year of the population data.
- **Sex**: The gender (male/female).
- **Age**: The age group or specific age.
- **Value**: Count of the population in the given category.
- **gss_code_ward**: Geocode for the ward.
- **ward_name**: Name of the ward.

### Components Column:
This column includes specific population-related metrics:
- **net-flow**: Population migration inflow minus migration outflow.
- **popn**: Total population.
- **birth**: Number of births.
- **deaths**: Number of deaths.



---

## 🛠️ **Structure**
1. [**Load and Preprocess** the population projections dataset.](#load-and-preprocess)
2. [**Define Utility Functions** for effective use.](#define-utility-functions)
3. [**Process Data**](#process-data)
4. [**Basic Checks** on the dataset.](#basic-checks)
5. [**Population Consistency Over Time** for each component.](#population-consistency-over-time)
6. [**Total Population Outliers**](#total-population-outliers)
7. [**Gender Outliers**](#gender-outliers)
8. [**Key Visualisations**](#key-visualisations)
9. [**Collate Outliers** to determine key outlier rows.](#collate-outliers)



## Load Data
This section will cover how to load and preprocess the dataset.

---


In [1]:
import pandas as pd
import numpy as np
import pyreadr
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
from typing import Dict


In [2]:
os.chdir(r'C:\Users\user\Documents\population_data\combined_10yr_central_fert')
combined_10yr_fert = pd.read_csv('combined_10yr_central_fert.csv').iloc[:, 1:]

C:\Users\user\AppData\Local\Temp\ipykernel_24332\1627369184.py:2: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_10yr_fert = pd.read_csv('combined_10yr_central_fert.csv').iloc[:, 1:]


In [3]:
combined_10yr_fert

,gss_code,la_name,year,sex,age,value,component,gss_code_ward,ward_name
0,E09000001,City of London,2012.0,female,0.0,32.0,births,NaN,NaN
1,E09000001,City of London,2012.0,male,0.0,24.0,births,NaN,NaN
2,E09000001,City of London,2013.0,female,0.0,36.0,births,NaN,NaN
3,E09000001,City of London,2013.0,male,0.0,35.0,births,NaN,NaN
4,E09000001,City of London,2014.0,female,0.0,26.0,births,NaN,NaN
...,...,...,...,...,...,...,...,...,...
15428057,E09000033,Westminster,2050.0,male,86.0,16.9,popn,E05013809,Westbourne
15428058,E09000033,Westminster,2050.0,male,87.0,13.3,popn,E05013809,Westbourne
15428059,E09000033,Westminster,2050.0,male,88.0,11.0,popn,E05013809,Westbourne
15428060,E09000033,Westminster,2050.0,male,89.0,9.9,popn,E05013809,Westbourne


## Define Utility Functions
Define utility functions that will be used for various parts of the analysis.

---

In [4]:
def view_descriptive_statistics(df, columns):
    """
    Calculate descriptive statistics, including mean, median, and mode, for specified columns in a DataFrame.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame.
    columns (list): List of columns for which to calculate the statistics.
    
    Returns:
    pd.DataFrame: DataFrame containing the descriptive statistics including median and mode.
    """
    # Get descriptive statistics using describe()
    descriptive_stats = df[columns].describe()

    # Calculate median for each column
    median = df[columns].median()

    # Calculate mode for each column (in case of multiple modes, take the first one)
    mode = df[columns].mode().iloc[0]

    # Add median and mode to the descriptive statistics DataFrame
    descriptive_stats.loc['median'] = median
    descriptive_stats.loc['mode'] = mode

    # Return the combined descriptive statistics
    return descriptive_stats

In [5]:
def create_age_bins(df, age_column='age', bins=None, labels=None):
    """
    Create age bins for the specified age column in the given DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the age data.
    age_column (str): The name of the column containing age data. Default is 'age'.
    bins (list): A list of bin edges for categorizing ages. Default is None.
    labels (list): A list of labels for the bins. Default is None.

    Returns:
    pd.DataFrame: The DataFrame with a new 'age' column containing binned age data.
    """
    
    # If bins and labels are not provided, set default values
    if bins is None:
        bins = [-1, 18, 30, 40, 50, 60, 70, 80, 89, 90]
    
    if labels is None:
        labels = ['0-18', '19-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-89', '90+']
    
    # Create a copy of the original DataFrame to avoid modifying it directly
    binned_df = df.copy()
    
    # Create age bins
    binned_df[age_column] = pd.cut(binned_df[age_column], bins=bins, labels=labels)
    
    return binned_df

# Example usage:
# combined_10yr_fert_agebins = create_age_bins(combined_10yr_fert)


In [114]:
def calculate_zscores_and_find_outliers(df, component_columns, handle_inf=True, Geography='borough', z_score_threshold=2, For_population_totals=False, population_analysis_type='cross-sectional'):
    """
    Computes z-scores or robust z-scores (depending on distribution) for the respective columns,
    and returns DataFrames containing outliers for either the component columns (handle_inf=True) 
    or the percentage change columns (handle_inf=False).

    Parameters:
    df (pd.DataFrame): The input DataFrame containing population data component value columns.
    component_columns (str or list): A single column name or a list of column names to be analysed.
    handle_inf (bool): If True, uses the component columns to determine outliers. 
                       If False, uses percentage change columns to determine outliers.
    Geography (str): Specifies whether to group by 'borough' (using 'gss_code') or 'ward' (using 'gss_code_ward').
                     Default is 'borough'.
    For_population_totals (bool): If True, calculates total population sums and percentage changes before proceeding 
                                  to z-score and outlier analysis.
    population_analysis_type (str): Specifies whether to do 'cross-sectional' or 'temporal' analysis for population totals.
                                    Default is 'cross-sectional'.
    z_score_threshold (float or int): The threshold to consider as an outlier based on the z-score. Default is 2.

    Returns:
    dict: A dictionary containing DataFrames with outliers for each respective column based on the z-score threshold.
    """
    
    # If a single column name is provided as a string, convert it to a list
    if isinstance(component_columns, str):
        component_columns = [component_columns]

    outliers_dict = {}
    z_score_type = {}  # Dictionary to store which method was used

    # Grouping and pivot based on the Geography parameter
    if Geography == 'borough':
        geo_column = 'gss_code'
    elif Geography == 'ward':
        geo_column = 'gss_code_ward'
    else:
        raise ValueError("Geography must be either 'borough' or 'ward'.")

    # Automatically create pct_change_columns based on component_columns
    pct_change_columns = [f"{col}_pct_change" for col in component_columns]

    #Calculate the percentage change for the component columns
    df[pct_change_columns] = df.groupby([geo_column, 'sex', 'age'])[component_columns].pct_change().abs()
    #pivoted = df.pivot_table(index=[geo_column, 'sex', 'age', 'year'], columns='component', values='value').reset_index()

    # If For_population_totals is True, calculate population totals and percentage changes
    if For_population_totals:
        # Filter the DataFrame to only keep rows where the 'component' is 'popn'
        # popn_df = pivoted[pivoted['component'] == 'popn']
        
        # Pivot the data to get 'popn' values by geography, sex, age, and year
        # pivoted = popn_df.pivot_table(index=[geo_column, 'sex', 'age', 'year'], columns='component', values='value').reset_index()
        
        # Ensure that there's a 'popn' column in the resulting DataFrame
        if 'popn' in df.columns:
            df['popn'] = df['popn']  # Extract the 'popn' column
        else:
            raise ValueError("The 'popn' column is missing after pivoting.")

        # Group by geography and year, and sum the population values, create total population dfs for crossectional and temporal analysis change
        population_sum = df.groupby([geo_column, 'year'])['popn'].sum().reset_index()

        population_sum_time = population_sum.copy()
        population_sum_cross = population_sum.copy()

        if population_analysis_type == 'temporal':
            # Temporally: Calculate the population change over the years for each gss_code or ward
            population_sum_time['popn_pct_change_temporal'] = population_sum_time.groupby(geo_column)['popn'].pct_change() * 100
        elif population_analysis_type == 'cross-sectional':
            # Cross-sectionally: Compare the population between different gss_code or wards for the same year and compare to the mean
            population_sum_cross['popn_mean'] = population_sum_cross.groupby('year')['popn'].transform('mean')
            population_sum_cross['popn_pct_change_cross'] = ((population_sum_cross['popn'] - population_sum_cross['popn_mean']) / population_sum_cross['popn_mean']) * 100
        else:
            raise ValueError("population_analysis_type must be either 'cross-sectional' or 'temporal'.")

    # Now, decide how to determine outliers based on handle_inf (handling the infinite values e.i. no percentage change value) and pct_change_columns
    if handle_inf:
        # Outliers based on component columns value (not percentage change)
        for comp_col, pct_change_col in zip(component_columns, pct_change_columns):
            if pct_change_col in df.columns:
                # Filter rows where percentage change columns have inf or -inf
                df_filtered = df.replace([np.inf, -np.inf], np.nan).dropna(subset=[comp_col, pct_change_col])

                if not df_filtered.empty:
                    # Check if the column is normally distributed using skewness
                    skewness = df_filtered[comp_col].skew()

                    if abs(skewness) < 0.5:  # If skewness is less than 0.5, use normal Z-score
                        df_filtered['z_score'] = stats.zscore(df_filtered[comp_col])
                        outliers = df_filtered[df_filtered['z_score'].abs() > z_score_threshold]  # Z-score > threshold
                        z_score_type[comp_col] = 'Normal Z-Score'
                        print(f"{comp_col} used Normal Z-Score.")
                    else:
                        # Use Robust Z-score (based on median and MAD) for non-normal distribution
                        median = df_filtered[comp_col].median()
                        mad = stats.median_abs_deviation(df_filtered[comp_col])
                        df_filtered['robust_z_score'] = (df_filtered[comp_col] - median) / mad
                        outliers = df_filtered[df_filtered['robust_z_score'].abs() > z_score_threshold]  # Robust Z-score > threshold
                        z_score_type[comp_col] = 'Robust Z-Score'
                        print(f"{comp_col} used Robust Z-Score.")

                    # Store the outliers for this component column
                    outliers_dict[comp_col] = outliers
                else:
                    outliers_dict[comp_col] = pd.DataFrame()  # Return empty DataFrame if no rows found
            else:
                print(f"{comp_col} does not exist in DataFrame")

    else:
        # Outliers based on pct_change_columns
        for pct_change_col in pct_change_columns:
            # For population totals choose the correct DataFrame (from above) based on the selected column
            if population_analysis_type == 'temporal' and 'popn_pct_change_temporal' in pct_change_col:
                df_filtered = population_sum_time  # Use the temporal population data
            elif population_analysis_type == 'cross-sectional' and 'popn_pct_change_cross' in pct_change_col:
                df_filtered = population_sum_cross  # Use the cross-sectional population data
            else:
                df_filtered = df  # Default to the original df if other percentage columns are provided

            #pct column for component column zscores
            if pct_change_col in df_filtered.columns:
                # Replace inf and -inf with NaN and work on the entire DataFrame after cleaning
                df_filtered = df_filtered.replace([np.inf, -np.inf], np.nan).dropna(subset=[pct_change_col])

                if not df_filtered.empty:
                    # Check if the column is normally distributed using skewness
                    skewness = df_filtered[pct_change_col].skew()

                    if abs(skewness) < 0.5:  # If skewness is less than 0.5, use normal Z-score
                        df_filtered['z_score'] = stats.zscore(df_filtered[pct_change_col])
                        outliers = df_filtered[df_filtered['z_score'].abs() > z_score_threshold]  # Z-score > threshold
                        z_score_type[pct_change_col] = 'Normal Z-Score'
                        print(f"{pct_change_col} used Normal Z-Score.")
                    else:
                        # Use Robust Z-score (based on median and MAD) for non-normal distribution
                        median = df_filtered[pct_change_col].median()
                        mad = stats.median_abs_deviation(df_filtered[pct_change_col])
                        df_filtered['robust_z_score'] = (df_filtered[pct_change_col] - median) / mad
                        outliers = df_filtered[df_filtered['robust_z_score'].abs() > z_score_threshold]  # Robust Z-score > threshold
                        z_score_type[pct_change_col] = 'Robust Z-Score'
                        print(f"{pct_change_col} used Robust Z-Score.")

                    # Store the outliers for this percentage change column
                    outliers_dict[pct_change_col] = outliers
                else:
                    outliers_dict[pct_change_col] = pd.DataFrame()  # Return empty DataFrame if no rows found
            else:
                print(f"{pct_change_col} does not exist in DataFrame")

    # Sort each DataFrame in outliers_dict by 'z_score' or 'robust_z_score' in descending order in one line
    for key in outliers_dict.keys():
        outliers_dict[key] = outliers_dict[key].sort_values(by='z_score' if 'z_score' in outliers_dict[key].columns else 'robust_z_score', ascending=False)

    return outliers_dict  # Returning z_score_type for further use if needed


In [7]:
def gender_outliers(df, component_columns, geography='borough', outlier_std={'births': 2, 'deaths': 5, 'netflow': 2, 'popn': 5}):
    """
    Processes gender data for either wards or boroughs, and calculates outliers for specified components.
    
    Parameters:
    - df: pandas DataFrame containing the raw data
    - component_columns: list or single component column name(s) for which ratios and outliers need to be calculated
    - geography: str, either 'ward' or 'borough', default is 'borough'
    - outlier_std: dict specifying how many standard deviations to use for each component's threshold calculation.
    
    Returns:
    - outliers_dict: dictionary of outlier DataFrames for each component
    """

    # Check geography type and set index columns accordingly
    if geography == 'ward':
        geo_col = 'gss_code_ward'
    else:
        geo_col = 'gss_code'
    
    # Step 1: Create the pivot table
    gender_pivot = df.pivot_table(
        index=[geo_col, 'year', 'age', 'component'],  # Geography column and other grouping columns
        columns='sex',                                # Columns for sex (male, female)
        values='value',                               # Values (count of males and females)
        aggfunc='sum'                                 # Aggregation function (sum)
    ).reset_index()
    
    # Step 2: Calculate the ratio of females to males
    gender_pivot['ratio_female_to_male'] = gender_pivot['female'] / gender_pivot['male']
    
    # Handle division by zero and missing values
    gender_pivot['ratio_female_to_male'].replace([float('inf'), -float('inf')], pd.NA, inplace=True)
    gender_pivot['ratio_female_to_male'].fillna(np.nan, inplace=True)
    
    # Step 3: Pivot again to spread component values into columns
    gender_pivot = gender_pivot.pivot(
        index=[geo_col, 'year', 'age'], 
        columns='component', 
        values='ratio_female_to_male'
    ).reset_index()
    
    # Step 4: Convert the specified component columns to numeric
    for component in component_columns:
        gender_pivot[component] = pd.to_numeric(gender_pivot[component], errors='coerce')
    
    # Step 5: Calculate mean and standard deviation for each component
    means = {}
    stds = {}
    for component in component_columns:
        means[component] = gender_pivot[component].mean()
        stds[component] = gender_pivot[component].std()
    
    # Step 6: Set thresholds for outliers using mean ± specified standard deviations
    thresholds = {}
    for component in component_columns:
        high_threshold = means[component] + outlier_std.get(component, 2) * stds[component]
        low_threshold = means[component] - outlier_std.get(component, 2) * stds[component]
        thresholds[component] = (low_threshold, high_threshold)
    
    # Step 7: Identify outliers for each component
    outliers_dict = {}
    for component in component_columns:
        low_threshold, high_threshold = thresholds[component]
        outliers = gender_pivot[(gender_pivot[component] > high_threshold) | (gender_pivot[component] < low_threshold)]
        outliers_dict[component] = outliers.reset_index(drop=True)
    
    return outliers_dict


In [8]:
def tally_outliers(outlier_dfs: Dict[str, pd.DataFrame], 
                   tally_by_age: bool = False, 
                   geography: str = 'ward') -> pd.DataFrame:
    """
    Tally occurrences of the same 'gss_code_ward' or 'gss_code', 
    'year', and optionally 'age' across multiple DataFrames.

    Parameters:
    ----------
    outlier_dfs : Dict[str, pd.DataFrame]
        A dictionary where keys are DataFrame names and values are the DataFrames themselves.
        Each DataFrame should contain 'gss_code_ward' or 'gss_code', 'year', and optionally 'age' columns.

    tally_by_age : bool, optional
        If True, the function will tally occurrences based on 'gss_code_ward' or 'gss_code', 'year', and 'age'.
        If False, it will tally based only on 'gss_code_ward' or 'gss_code' and 'year'. Default is False.

    geography : str, optional
        Specifies the geographical code to use. Set to 'borough' to use 'gss_code' 
        or 'ward' to use 'gss_code_ward'. Default is 'ward'.

    Returns:
    -------
    pd.DataFrame
        A DataFrame with counts of occurrences for each combination of 
        'gss_code_ward' or 'gss_code', 'year', and 'age' (if applicable), 
        along with a total count across all DataFrames.
    """
    
    # Validate input
    if not isinstance(outlier_dfs, dict):
        raise ValueError("Expected 'outlier_dfs' to be a dictionary of DataFrames.")
    
    if geography not in ['borough', 'ward']:
        raise ValueError("geography must be either 'borough' or 'ward'.")

    # Initialise an empty DataFrame for the combined tally
    combined_tally = pd.DataFrame()

    # Define the geography column based on the geography argument
    geo_column = 'gss_code' if geography == 'borough' else 'gss_code_ward'

    # Loop through each DataFrame in the outlier_dfs dictionary
    for df_name, df in outlier_dfs.items():
        if not isinstance(df, pd.DataFrame):
            raise ValueError(f"Expected '{df_name}' to be a pandas DataFrame.")
        
        # Ensure 'age' is consistently typed if tally_by_age is True
        if tally_by_age and 'age' in df.columns:
            df['age'] = df['age'].astype(str)

        # Select the required columns based on the tally_by_age flag
        group_cols = [geo_column, 'year'] + (['age'] if tally_by_age and 'age' in df.columns else [])

        # Ensure selected columns exist in the DataFrame
        missing_cols = [col for col in group_cols if col not in df.columns]
        if missing_cols:
            raise KeyError(f"The DataFrame '{df_name}' is missing columns: {missing_cols}")

        # Remove duplicate rows based on the selected grouping columns
        subset = df[group_cols].drop_duplicates()
        
        # Add a count column for tallying occurrences
        subset['count'] = 1

        # Aggregate the counts by the grouping columns
        grouped = subset.groupby(group_cols).count().reset_index()

        # Rename the count column to indicate which DataFrame it came from
        grouped.rename(columns={'count': df_name}, inplace=True)

        # Merge the current tally with the combined tally
        if combined_tally.empty:
            combined_tally = grouped
        else:
            combined_tally = pd.merge(combined_tally, grouped, on=group_cols, how='outer')

    # Fill NaN values with 0 (indicating no occurrences in that DataFrame)
    combined_tally.fillna(0, inplace=True)

    # Calculate the total tally across all DataFrame columns
    tally_columns = combined_tally.columns.difference(group_cols)
    combined_tally['total'] = combined_tally[tally_columns].sum(axis=1)

    # Sort the DataFrame by the total count in descending order
    combined_tally.sort_values(by='total', ascending=False, inplace=True)

    return combined_tally

# Example usage:
# result_df = tally_outliers(outlier_dfs, tally_by_age=True, geography='borough')
# print(result_df)



## Proccess data

---

In [9]:
#split ward and borough dataframes
combined_10yr_fert_boroughs = combined_10yr_fert[combined_10yr_fert['gss_code_ward'].isna()]
combined_10yr_fert_ward = combined_10yr_fert[~combined_10yr_fert['gss_code_ward'].isna()]

In [10]:
#bin ages
combined_10yr_fert_agebins = create_age_bins(combined_10yr_fert)

In [11]:
#seperate components into columns
combined_10yr_fert_agebins_component_columns = combined_10yr_fert_agebins.pivot_table(index=['gss_code','gss_code_ward','sex', 'age','year'], columns='component', values='value').reset_index()

C:\Users\user\AppData\Local\Temp\ipykernel_24332\2449090281.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  combined_10yr_fert_agebins_component_columns = combined_10yr_fert_agebins.pivot_table(index=['gss_code','gss_code_ward','sex', 'age','year'], columns='component', values='value').reset_index()


## Basic Checks
Perform basic checks on the dataset, including checking for missing values, duplicates, and descriptive statistics.

---

In [12]:
#min and max year
def get_year_range(df):
    return df['year'].max(), df['year'].min()

In [13]:
#year ranges
print(get_year_range(combined_10yr_fert))
print(get_year_range(combined_10yr_fert_ward))
print(get_year_range(combined_10yr_fert_boroughs))

(2050.0, 2002.0)
(2050.0, 2011.0)
(2050.0, 2002.0)


##### missing values

In [14]:
missing_values = combined_10yr_fert.isnull().sum()
print("Missing values per column:\n", missing_values)

Missing values per column:
 gss_code              0
la_name               0
year                  0
sex                   0
age                   0
value                 0
component             0
gss_code_ward    771342
ward_name        771342
dtype: int64


#### duplicates

In [15]:
duplicates = combined_10yr_fert.duplicated().sum()
print("Number of duplicate rows:", duplicates)

Number of duplicate rows: 0


##### Descriptive data

In [16]:
combined_10yr_fert.describe()

,year,age,value
count,1.542806e+07,1.542806e+07,1.542806e+07
mean,2.030736e+03,4.483779e+01,4.879530e+01
std,1.143556e+01,2.635879e+01,2.227471e+02
min,2.002000e+03,0.000000e+00,-7.130000e+02
25%,2.021000e+03,2.200000e+01,0.000000e+00
50%,2.031000e+03,4.500000e+01,8.000000e-01
75%,2.041000e+03,6.800000e+01,4.790000e+01
max,2.050000e+03,9.000000e+01,5.922600e+03


##### Description by components

In [17]:
# Group by 'component' column
grouped = combined_10yr_fert.groupby('component')

# Apply describe to each group
described_groups = grouped.describe()

In [18]:
described_groups

year                                                   \
               count         mean        std     min      25%     50%   
component                                                               
births       55614.0  2031.000000  11.254730  2012.0  2021.00  2031.0   
deaths     5060874.0  2031.000000  11.254630  2012.0  2021.00  2031.0   
netflow    5120934.0  2030.712656  11.499358  2002.0  2021.00  2031.0   
popn       5190640.0  2030.500000  11.543397  2011.0  2020.75  2030.5   

                                  age        ...                  value  \
               75%     max      count  mean  ...   75%   max      count   
component                                    ...                          
births     2041.00  2050.0    55614.0   0.0  ...   0.0   0.0    55614.0   
deaths     2041.00  2050.0  5060874.0  45.0  ...  68.0  90.0  5060874.0   
netflow    2041.00  2050.0  5120934.0  45.0  ...  68.0  90.0  5120934.0   
popn       2040.25  2050.0  5190640.0  45.0  ...  68.0  90.0  5190640.0   

                                                                     
                 mean         std    min   25%   50%    75%     max  
component                                                            
births     162.293288  377.997791    0.0  59.9  81.6  110.9  3264.0  
deaths       0.885798    6.327541    0.0   0.0   0.1    0.6   825.2  
netflow     -0.182117   20.742161 -713.0  -2.3  -0.4    1.2  1938.0  
popn       142.610704  363.229346    0.0  45.0  71.2  103.6  5922.6  

[4 rows x 24 columns]

In [19]:
# List of percentage change columns
pct_change_columns = ['births_pct_change', 'deaths_pct_change', 'netflow_pct_change', 'popn_pct_change']
component_columns = ['births', 'deaths', 'netflow', 'popn'] 

# Call the function
descriptive_stats = view_descriptive_statistics(combined_10yr_fert_agebins_component_columns, component_columns)

# Display the descriptive statistics
print(descriptive_stats)

component        births         deaths        netflow           popn
count      53040.000000  477360.000000  477360.000000  489600.000000
mean          85.084712       1.491097      -0.239197      70.710902
std           36.143086       4.131203       3.288641      44.500500
min            0.000000       0.000000     -50.050000       0.000000
25%           59.100000       0.021053      -1.220000      37.880000
50%           79.600000       0.220000      -0.470000      65.326316
75%          106.000000       0.910000       0.270000      95.850000
max          615.900000      87.300000      97.825000     737.090000
median        79.600000       0.220000      -0.470000      65.326316
mode          60.000000       0.000000      -0.500000      13.000000


#### check for negative values in columns

In [20]:
# Checking for negative values and extremely high values
negative_values = combined_10yr_fert[combined_10yr_fert['value'] < 0]
print('components with negative values:', negative_values['component'].unique())

components with negative values: ['netflow']


#### check age range

In [21]:
#print true if max age is 90 and min age is 0
print('Is max age is 90 and min age is 0:', (combined_10yr_fert['age'].max() == 90) & (combined_10yr_fert['age'].min() == 0))

Is max age is 90 and min age is 0: True


## Population Consistency Over Time

---

In [22]:
#use binned ages this will even out large flunctions between age group where the are likely to be unusally high e.i. 18 year olds moving to university
combined_10yr_fert_agebins_component_columns

component,gss_code,gss_code_ward,sex,age,year,births,deaths,netflow,popn
0,E09000001,E09000001,female,0-18,2011.0,NaN,NaN,NaN,19.421053
1,E09000001,E09000001,female,0-18,2012.0,32.0,0.0,0.631579,20.368421
2,E09000001,E09000001,female,0-18,2013.0,36.0,0.0,-1.526316,19.947368
3,E09000001,E09000001,female,0-18,2014.0,26.0,0.0,0.052632,19.473684
4,E09000001,E09000001,female,0-18,2015.0,30.0,0.0,-0.210526,19.578947
...,...,...,...,...,...,...,...,...,...
489595,E09000033,E05013809,male,90+,2046.0,NaN,7.8,0.900000,20.000000
489596,E09000033,E05013809,male,90+,2047.0,NaN,8.1,0.900000,21.000000
489597,E09000033,E05013809,male,90+,2048.0,NaN,8.5,0.900000,22.200000
489598,E09000033,E05013809,male,90+,2049.0,NaN,8.7,0.900000,23.200000


In [23]:
outliers_dict_borough, z_score_type = calculate_zscores_and_find_outliers(
    combined_10yr_fert_agebins_component_columns, 
    ['births', 'deaths', 'netflow', 'popn'], 
    handle_inf=False, 
    Geography='borough',
    z_score_threshold=3,
    For_population_totals=False
    )

C:\Users\user\AppData\Local\Temp\ipykernel_24332\1801048556.py:43: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[pct_change_columns] = df.groupby([geo_column, 'sex', 'age'])[component_columns].pct_change().abs()
C:\Users\user\AppData\Local\Temp\ipykernel_24332\1801048556.py:43: FutureWarning: The default fill_method='ffill' in DataFrameGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df[pct_change_columns] = df.groupby([geo_column, 'sex', 'age'])[component_columns].pct_change().abs()


births_pct_change used Robust Z-Score.
deaths_pct_change used Robust Z-Score.
netflow_pct_change used Robust Z-Score.
popn_pct_change used Robust Z-Score.


In [24]:
outliers_dict_ward, z_score_type = calculate_zscores_and_find_outliers(
    combined_10yr_fert_agebins_component_columns, 
    ['births', 'deaths', 'netflow', 'popn'], 
    handle_inf=False, 
    Geography='ward',
    z_score_threshold=3,
    For_population_totals=False
    )

C:\Users\user\AppData\Local\Temp\ipykernel_24332\1801048556.py:43: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[pct_change_columns] = df.groupby([geo_column, 'sex', 'age'])[component_columns].pct_change().abs()
C:\Users\user\AppData\Local\Temp\ipykernel_24332\1801048556.py:43: FutureWarning: The default fill_method='ffill' in DataFrameGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df[pct_change_columns] = df.groupby([geo_column, 'sex', 'age'])[component_columns].pct_change().abs()


births_pct_change used Robust Z-Score.
deaths_pct_change used Robust Z-Score.
netflow_pct_change used Robust Z-Score.
popn_pct_change used Robust Z-Score.


In [25]:
outliers_dict_borough_inf_values, z_score_type = calculate_zscores_and_find_outliers(
    combined_10yr_fert_agebins_component_columns, 
    ['births', 'deaths', 'netflow', 'popn'], 
    handle_inf=True, 
    Geography='borough',
    z_score_threshold=3,
    For_population_totals=False
    )

C:\Users\user\AppData\Local\Temp\ipykernel_24332\1801048556.py:43: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[pct_change_columns] = df.groupby([geo_column, 'sex', 'age'])[component_columns].pct_change().abs()
C:\Users\user\AppData\Local\Temp\ipykernel_24332\1801048556.py:43: FutureWarning: The default fill_method='ffill' in DataFrameGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df[pct_change_columns] = df.groupby([geo_column, 'sex', 'age'])[component_columns].pct_change().abs()


births used Robust Z-Score.
deaths used Robust Z-Score.
netflow used Robust Z-Score.
popn used Robust Z-Score.


In [26]:
outliers_dict_ward_inf_values, z_score_type = calculate_zscores_and_find_outliers(
    combined_10yr_fert_agebins_component_columns, 
    ['births', 'deaths', 'netflow', 'popn'], 
    handle_inf=True, 
    Geography='ward',
    z_score_threshold=3,
    For_population_totals=False)

C:\Users\user\AppData\Local\Temp\ipykernel_24332\1801048556.py:43: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[pct_change_columns] = df.groupby([geo_column, 'sex', 'age'])[component_columns].pct_change().abs()
C:\Users\user\AppData\Local\Temp\ipykernel_24332\1801048556.py:43: FutureWarning: The default fill_method='ffill' in DataFrameGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df[pct_change_columns] = df.groupby([geo_column, 'sex', 'age'])[component_columns].pct_change().abs()


births used Robust Z-Score.
deaths used Robust Z-Score.
netflow used Robust Z-Score.
popn used Robust Z-Score.


In [27]:
births_pct_change_borough_outlier_df = outliers_dict_borough['births_pct_change']
deaths_pct_change_borough_outlier_df = outliers_dict_borough['deaths_pct_change']
netflow_pct_change_borough_outlier_df = outliers_dict_borough['netflow_pct_change']
popn_pct_change_borough_outlier_df = outliers_dict_borough['popn_pct_change']

In [28]:
births_pct_change_ward_outlier_df = outliers_dict_ward['births_pct_change']
deaths_pct_change_ward_outlier_df = outliers_dict_ward['deaths_pct_change']
netflow_pct_change_ward_outlier_df = outliers_dict_ward['netflow_pct_change']
popn_pct_change_ward_outlier_df = outliers_dict_ward['popn_pct_change']

In [29]:
births_pct_change_borough_inf_outlier_df = outliers_dict_borough_inf_values['births']
deaths_pct_change_borough_inf_outlier_df = outliers_dict_borough_inf_values['deaths']
netflow_pct_change_borough_inf_outlier_df = outliers_dict_borough_inf_values['netflow']
popn_pct_change_borough_inf_outlier_df = outliers_dict_borough_inf_values['popn']

In [30]:
births_pct_change_ward_inf_outlier_df = outliers_dict_ward_inf_values['births']
deaths_pct_change_ward_inf_outlier_df = outliers_dict_ward_inf_values['deaths']
netflow_pct_change_ward_inf_outlier_df = outliers_dict_ward_inf_values['netflow']   
popn_pct_change_ward_inf_outlier_df = outliers_dict_ward_inf_values['popn']

## Total Population Outliers
#### Detect total outliers using Z-scores and Robust Z-scores, and perform cross-sectional and temporal comparisons.

#### Using total population (popn) perform cross-sectional comparisons: Examine changes between boroughs and wards totals for a given year.
#### Conduct temporal comparisons: Measure percentage changes between year total for both boroughs and wards

#### Total population per geographical boundary
#### Total population per year
#### i.e.
#### groupby('gss_code')['popn']
#### groupby('year')['popn']
---

In [31]:
combined_10yr_fert_agebins_component_columns

component,gss_code,gss_code_ward,sex,age,year,births,deaths,netflow,popn,births_pct_change,deaths_pct_change,netflow_pct_change,popn_pct_change
0,E09000001,E09000001,female,0-18,2011.0,NaN,NaN,NaN,19.421053,NaN,NaN,NaN,NaN
1,E09000001,E09000001,female,0-18,2012.0,32.0,0.0,0.631579,20.368421,NaN,NaN,NaN,0.048780
2,E09000001,E09000001,female,0-18,2013.0,36.0,0.0,-1.526316,19.947368,0.125000,NaN,3.416667,0.020672
3,E09000001,E09000001,female,0-18,2014.0,26.0,0.0,0.052632,19.473684,0.277778,NaN,1.034483,0.023747
4,E09000001,E09000001,female,0-18,2015.0,30.0,0.0,-0.210526,19.578947,0.153846,NaN,5.000000,0.005405
...,...,...,...,...,...,...,...,...,...,...,...,...,...
489595,E09000033,E05013809,male,90+,2046.0,NaN,7.8,0.900000,20.000000,NaN,0.040000,0.000000,0.063830
489596,E09000033,E05013809,male,90+,2047.0,NaN,8.1,0.900000,21.000000,NaN,0.038462,0.000000,0.050000
489597,E09000033,E05013809,male,90+,2048.0,NaN,8.5,0.900000,22.200000,NaN,0.049383,0.000000,0.057143
489598,E09000033,E05013809,male,90+,2049.0,NaN,8.7,0.900000,23.200000,NaN,0.023529,0.000000,0.045045


In [32]:
total_pop_temporal_outliers_ward, z_score_type = calculate_zscores_and_find_outliers(
    combined_10yr_fert_agebins_component_columns, 
    ['popn'], 
    handle_inf=False, 
    Geography='ward',
    For_population_totals=True,
    population_analysis_type='temporal'
    )

C:\Users\user\AppData\Local\Temp\ipykernel_24332\1801048556.py:43: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[pct_change_columns] = df.groupby([geo_column, 'sex', 'age'])[component_columns].pct_change().abs()


popn_pct_change used Robust Z-Score.


In [33]:
total_pop_temporal_outliers_ward_df  = total_pop_temporal_outliers_ward['popn_pct_change']

In [34]:
total_pop_cross_sectional_outliers_ward, z_score_type = calculate_zscores_and_find_outliers(
    combined_10yr_fert_agebins_component_columns, 
    ['popn'], 
    handle_inf=False, 
    Geography='ward',
    For_population_totals=True,
    population_analysis_type='cross-sectional'
    )

C:\Users\user\AppData\Local\Temp\ipykernel_24332\1801048556.py:43: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[pct_change_columns] = df.groupby([geo_column, 'sex', 'age'])[component_columns].pct_change().abs()


popn_pct_change used Robust Z-Score.


In [35]:
total_pop_cross_sectional_outliers_ward_df = total_pop_cross_sectional_outliers_ward['popn_pct_change']

## Gender Outliers
Investigate gender outliers, focusing on abnormal gender ratios and adjusting thresholds as needed.

---

In [36]:
#single year age 
component_columns = ['births', 'deaths', 'netflow', 'popn']
gender_outlier_dictionary = gender_outliers(combined_10yr_fert, component_columns, geography='ward', outlier_std={'births': 2, 'deaths': 5, 'netflow': 2, 'popn': 5})

C:\Users\user\AppData\Local\Temp\ipykernel_24332\989429346.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gender_pivot['ratio_female_to_male'].replace([float('inf'), -float('inf')], pd.NA, inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_24332\989429346.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', 

In [37]:
#binned age
component_columns = ['births', 'deaths', 'netflow', 'popn']
gender_outlier_dictionary = gender_outliers(combined_10yr_fert_agebins, component_columns, geography='ward', outlier_std={'births': 2, 'deaths': 5, 'netflow': 2, 'popn': 5})

C:\Users\user\AppData\Local\Temp\ipykernel_24332\989429346.py:22: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  gender_pivot = df.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_24332\989429346.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gender_pivot['ratio_female_to_male'].replace([float('inf'), -float('inf')], pd.NA, inplace=True)
C:\Users\user\Ap

In [38]:
gender_outlier_dictionary
births_gender_outliers_df = gender_outlier_dictionary['births']
deaths_gender_outliers_df = gender_outlier_dictionary['deaths']
netflow_gender_outliers_df = gender_outlier_dictionary['netflow']
popn_gender_outliers_df = gender_outlier_dictionary['popn']

## Key Visualisations
Visualise important trends in the dataset, including distribution by components, age groups, yearly totals, and population pyramids.

---


#### yearly totals by component visulation

In [39]:
yearly_totals = combined_10yr_fert.groupby(['year','component'])['value'].sum().reset_index()
print("Yearly population totals by gss_code and year:\n", yearly_totals)

Yearly population totals by gss_code and year:
        year component       value
0    2002.0   netflow      7036.6
1    2003.0   netflow    -37021.3
2    2004.0   netflow    -16399.9
3    2005.0   netflow     21802.2
4    2006.0   netflow      9229.4
..      ...       ...         ...
162  2049.0      popn  19763868.6
163  2050.0    births    226639.0
164  2050.0    deaths    137879.9
165  2050.0   netflow    -57727.1
166  2050.0      popn  19794455.5

[167 rows x 3 columns]


In [40]:

# Create a bar graph using Plotly Express
fig = px.line(
    yearly_totals, 
    x='year', 
    y='value', 
    color='component', 
    markers=True,
    title="Yearly Totals by Component Over the Years",
    labels={'value': 'Total Value', 'year': 'Year'},
    
)

# Update layout for better appearance
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Total Value',
    legend_title='Component',
    width=900,
    height=600
)

# Show the figure
fig.show()

#### population pyramid

In [41]:
#for unit age
combined_10yr_fert_popn = combined_10yr_fert[combined_10yr_fert['component'] == 'popn']
population_pyramids_unit_age = combined_10yr_fert_popn.copy()
population_pyramids_unit_age = population_pyramids_unit_age[~population_pyramids_unit_age['gss_code_ward'].isna()]

In [42]:
population_pyramids_unit_age

,gss_code,la_name,year,sex,age,value,component,gss_code_ward,ward_name
10477662,E09000001,City of London,2011.0,female,0.0,37.0,popn,E09000001,City of London
10477663,E09000001,City of London,2011.0,female,1.0,34.0,popn,E09000001,City of London
10477664,E09000001,City of London,2011.0,female,2.0,28.0,popn,E09000001,City of London
10477665,E09000001,City of London,2011.0,female,3.0,18.0,popn,E09000001,City of London
10477666,E09000001,City of London,2011.0,female,4.0,21.0,popn,E09000001,City of London
...,...,...,...,...,...,...,...,...,...
15428057,E09000033,Westminster,2050.0,male,86.0,16.9,popn,E05013809,Westbourne
15428058,E09000033,Westminster,2050.0,male,87.0,13.3,popn,E05013809,Westbourne
15428059,E09000033,Westminster,2050.0,male,88.0,11.0,popn,E05013809,Westbourne
15428060,E09000033,Westminster,2050.0,male,89.0,9.9,popn,E05013809,Westbourne


In [43]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='location-dropdown',
        options=[{'label': loc, 'value': loc} for loc in population_pyramids_unit_age['la_name'].unique()] + [{'label': 'London Total (All LAs)', 'value': 'London Total (All LAs)'}],
        value=population_pyramids_unit_age['la_name'].unique()[0]
    ),
    dcc.Dropdown(
        id='ward-dropdown'
    ),
    dcc.Graph(id='population-pyramid')
])

@app.callback(
    Output('ward-dropdown', 'options'),
    Output('ward-dropdown', 'value'),
    Input('location-dropdown', 'value')
)
def update_ward_dropdown(selected_location):
    if selected_location == 'London Total (All LAs)':
        # If "London Total (All LAs)" is selected, disable the ward dropdown
        return [{'label': 'All Wards', 'value': 'All Wards'}], 'All Wards'
    else:
        # Filter data for the selected location to get ward names
        filtered_data = population_pyramids_unit_age[population_pyramids_unit_age['la_name'] == selected_location]
        
        # Get unique ward names for the selected location
        ward_options = [{'label': ward, 'value': ward} for ward in filtered_data['ward_name'].unique()]
        
        # Add 'All Wards' option
        ward_options.insert(0, {'label': 'All Wards', 'value': 'All Wards'})
        
        # Return the options and set the default value to 'All Wards'
        return ward_options, 'All Wards'

@app.callback(
    Output('population-pyramid', 'figure'),
    Input('location-dropdown', 'value'),
    Input('ward-dropdown', 'value')
)
def update_pyramid(selected_location, selected_ward):
    if selected_location == 'London Total (All LAs)':
        # Combine data for all locations
        filtered_data = population_pyramids_unit_age.copy()
        title = 'Population Pyramid for London Total (All LAs)'
    elif selected_ward == 'All Wards':
        # Combine data for all wards in the selected location
        filtered_data = population_pyramids_unit_age[population_pyramids_unit_age['la_name'] == selected_location]
        title = f'Population Pyramid for {selected_location} - All Wards'
    else:
        # Filter data for the selected location and ward
        filtered_data = population_pyramids_unit_age[
            (population_pyramids_unit_age['la_name'] == selected_location) & 
            (population_pyramids_unit_age['ward_name'] == selected_ward)
        ]
        title = f'Population Pyramid for {selected_location} - {selected_ward}'

    # Negate female population values to create a pyramid
    filtered_data['value'] = filtered_data.apply(lambda row: -row['value'] if row['sex'] == 'female' else row['value'], axis=1)

    # Create a plotly express bar chart with animation
    fig = px.bar(
        filtered_data,
        x='value',
        y='age',
        color='sex',
        animation_frame='year',
        orientation='h',
        title=title,
        labels={'value': 'Population', 'age': 'Age'},
        color_discrete_map={'male': 'blue', 'female': 'pink'},
        height=600,
        range_x=[-filtered_data['value'].max()*1.2, filtered_data['value'].max()*1.2], #need to keep x-axis consistent over time for comparison
        hover_data={'ward_name': True}
    )

    # Update layout for better appearance
    fig.update_layout(
        barmode='relative',
        xaxis_title='Population',
        yaxis_title='Age',
        showlegend=True,
        width=800,
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=1223)

C:\Users\user\AppData\Local\Temp\ipykernel_24332\668186827.py:60: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\user\AppData\Local\Temp\ipykernel_24332\668186827.py:60: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\user\AppData\Local\Temp\ipykernel_24332\668186827.py:60: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d


## Collate Outliers
Summarize and compile all detected outliers for further analysis.

---

In [44]:
# Filter the global variables to find dataframes with 'outlier' in their name
outlier_dfs = {name: df for name, df in globals().items() if isinstance(df, pd.DataFrame) and 'outlier' in name.lower()}

# Display the name, columns, and length of each dataframe
for name, df in outlier_dfs.items():
    print(f"DataFrame Name: {name}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"Length: {len(df)}")
    print("\n" + "-"*50 + "\n")

DataFrame Name: births_pct_change_borough_outlier_df
Columns: ['gss_code', 'gss_code_ward', 'sex', 'age', 'year', 'births', 'deaths', 'netflow', 'popn', 'births_pct_change', 'deaths_pct_change', 'netflow_pct_change', 'popn_pct_change', 'robust_z_score']
Length: 16305

--------------------------------------------------

DataFrame Name: deaths_pct_change_borough_outlier_df
Columns: ['gss_code', 'gss_code_ward', 'sex', 'age', 'year', 'births', 'deaths', 'netflow', 'popn', 'births_pct_change', 'deaths_pct_change', 'netflow_pct_change', 'popn_pct_change', 'robust_z_score']
Length: 117296

--------------------------------------------------

DataFrame Name: netflow_pct_change_borough_outlier_df
Columns: ['gss_code', 'gss_code_ward', 'sex', 'age', 'year', 'births', 'deaths', 'netflow', 'popn', 'births_pct_change', 'deaths_pct_change', 'netflow_pct_change', 'popn_pct_change', 'robust_z_score']
Length: 138078

--------------------------------------------------

DataFrame Name: popn_pct_change_bo

In [45]:
#only ward data dtatframes
# Filter the global variables to find dataframes with 'outlier' in their name and exclude ones with 'borough'
outlier_dfs_wards = {name: df for name, df in globals().items() 
               if isinstance(df, pd.DataFrame) 
               and 'outlier' in name.lower() 
               and 'borough' not in name.lower()}

# Display the name, columns, and length of each dataframe
for name, df in outlier_dfs_wards.items():
    print(f"DataFrame Name: {name}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"Length: {len(df)}")
    print("\n" + "-"*50 + "\n")


DataFrame Name: births_pct_change_ward_outlier_df
Columns: ['gss_code', 'gss_code_ward', 'sex', 'age', 'year', 'births', 'deaths', 'netflow', 'popn', 'births_pct_change', 'deaths_pct_change', 'netflow_pct_change', 'popn_pct_change', 'robust_z_score']
Length: 15207

--------------------------------------------------

DataFrame Name: deaths_pct_change_ward_outlier_df
Columns: ['gss_code', 'gss_code_ward', 'sex', 'age', 'year', 'births', 'deaths', 'netflow', 'popn', 'births_pct_change', 'deaths_pct_change', 'netflow_pct_change', 'popn_pct_change', 'robust_z_score']
Length: 107953

--------------------------------------------------

DataFrame Name: netflow_pct_change_ward_outlier_df
Columns: ['gss_code', 'gss_code_ward', 'sex', 'age', 'year', 'births', 'deaths', 'netflow', 'popn', 'births_pct_change', 'deaths_pct_change', 'netflow_pct_change', 'popn_pct_change', 'robust_z_score']
Length: 129644

--------------------------------------------------

DataFrame Name: popn_pct_change_ward_outlie

In [46]:
# View age column in dfs
age_snippets = {df_name: df['age'].iloc[0] for df_name, df in outlier_dfs_wards.items()}
age_snippets

{'births_pct_change_ward_outlier_df': '0-18',
 'deaths_pct_change_ward_outlier_df': '19-30',
 'netflow_pct_change_ward_outlier_df': '0-18',
 'popn_pct_change_ward_outlier_df': '0-18',
 'births_pct_change_ward_inf_outlier_df': '0-18',
 'deaths_pct_change_ward_inf_outlier_df': '90+',
 'netflow_pct_change_ward_inf_outlier_df': '19-30',
 'popn_pct_change_ward_inf_outlier_df': '0-18',
 'total_pop_temporal_outliers_ward_df': '0-18',
 'total_pop_cross_sectional_outliers_ward_df': '0-18',
 'births_gender_outliers_df': '0-18',
 'deaths_gender_outliers_df': '90+',
 'netflow_gender_outliers_df': '71-80',
 'popn_gender_outliers_df': '90+'}

In [47]:
# Initialise an empty list to store relevant rows from all DataFrames
all_rows = []

# Loop through all outlier dataframes
for name, df in outlier_dfs_wards.items():
    # Check if 'gss_code' and 'year' columns exist
    if 'gss_code_ward' in df.columns and 'year' in df.columns:
        # Select the relevant columns: 'gss_code', 'year' and 'age' (if it exists)
        cols = ['gss_code_ward', 'year']
        if 'age' in df.columns:
            cols.append('age')

        # Append the relevant data from the current DataFrame to the list
        all_rows.append(df[cols])

# Concatenate all collected data into one DataFrame
if all_rows:
    combined_df = pd.concat(all_rows)

    # Group by 'gss_code', 'year', and 'age' (where applicable) and count occurrences
    tally_df = combined_df.groupby(cols).size().reset_index(name='count')

    # Sort the result by count in descending order
    tally_df = tally_df.sort_values(by='count', ascending=False)

    # Display the top rows of the tally DataFrame
    print(tally_df)
else:
    print("No relevant data found.")

C:\Users\user\AppData\Local\Temp\ipykernel_24332\469539181.py:21: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



       gss_code_ward    year    age  count
221211     E05014055  2030.0   0-18     17
221220     E05014055  2031.0   0-18     16
221247     E05014055  2034.0   0-18     16
125082     E05013694  2029.0   0-18     16
221238     E05014055  2033.0   0-18     16
...              ...     ...    ...    ...
127795     E05013701  2050.0  51-60      0
127794     E05013701  2050.0  41-50      0
127791     E05013701  2050.0   0-18      0
127787     E05013701  2049.0  61-70      0
122400     E05013687  2011.0   0-18      0

[244800 rows x 4 columns]


In [48]:
def tally_outliers(outlier_dfs, tally_by_age=False):
    # Initialize an empty dataframe to store the tally
    combined_tally = pd.DataFrame()
    
    # Loop through each dataframe in outlier_dfs
    for df_name, df in outlier_dfs.items():
        # Ensure 'age' is converted to string if tally_by_age is True
        if tally_by_age and 'age' in df.columns:
            df['age'] = df['age'].astype(str)
        
        # Select the columns based on whether tally_by_age is True or False
        if tally_by_age:
            subset = df[['gss_code_ward', 'year', 'age']].drop_duplicates()
        else:
            subset = df[['gss_code_ward', 'year']].drop_duplicates()
        
        # Create a column for the tally from the current dataframe
        subset['count'] = 1  # Each row represents one occurrence
        
        # Group by the subset to get a count of each combination
        grouped = subset.groupby(list(subset.columns[:-1])).count().reset_index()

        # Rename the 'count' column to correspond to the dataframe it came from
        grouped.rename(columns={'count': df_name}, inplace=True)
        
        # Merge the tally into the combined tally dataframe
        if combined_tally.empty:
            combined_tally = grouped
        else:
            combined_tally = pd.merge(combined_tally, grouped, 
                                      on=list(subset.columns[:-1]), 
                                      how='outer')
    
    # Fill NaN values with 0 (meaning that combination didn't appear in that df)
    combined_tally.fillna(0, inplace=True)
    
    # Add a 'total' column that sums across all dataframes
    tally_columns = combined_tally.columns[len(list(subset.columns[:-1])):]
    combined_tally['total'] = combined_tally[tally_columns].sum(axis=1)

    combined_tally['total'] = pd.to_numeric(combined_tally['total'])

    combined_tally = combined_tally.sort_values(by='total', ascending=False)
    
    # Return the final combined tally dataframe
    return combined_tally

# Example usage:
#result_df = tally_outliers(outlier_dfs, tally_by_age=True)
#print(result_df)



In [49]:
df_tally = tally_outliers(outlier_dfs_wards, tally_by_age=True)


In [52]:
df_tally

component,gss_code_ward,year,age,births_pct_change_ward_outlier_df,deaths_pct_change_ward_outlier_df,netflow_pct_change_ward_outlier_df,popn_pct_change_ward_outlier_df,births_pct_change_ward_inf_outlier_df,deaths_pct_change_ward_inf_outlier_df,netflow_pct_change_ward_inf_outlier_df,popn_pct_change_ward_inf_outlier_df,total_pop_temporal_outliers_ward_df,total_pop_cross_sectional_outliers_ward_df,births_gender_outliers_df,deaths_gender_outliers_df,netflow_gender_outliers_df,popn_gender_outliers_df,total
157252,E05014057,2021.0,90+,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,9.0
32039,E05011465,2022.0,90+,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,9.0
135075,E05013930,2022.0,90+,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,9.0
156680,E05014055,2030.0,0-18,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,9.0
131137,E05013915,2022.0,90+,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109263,E05013774,2045.0,90+,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
109264,E05013774,2046.0,41-50,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
109265,E05013774,2046.0,71-80,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
28775,E05011245,2045.0,31-40,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [51]:
# potential alterations:
# using single year of age vs binned ages 
# add weighting to final tally
# remove certain age groups like 90+ from outlier detection
# household data
# hmtl report generation


#### Household population

In [67]:
#read csv
household_data = pd.read_csv(r"C:/Users/user/Documents/population_data/2022_Identified_Capacity_15yr_high_fert/2022_Identified_Capacity_15yr_high_fert/csv/household_data.csv")

In [68]:
household_data

,Unnamed: 0,gss_code,la_name,gss_code_ward,ward_name,year,sex,age,popn,ce_popn,household_popn
0,0,E09000001,City of London,E09000001,City of London,2011.0,female,0.0,37.000000,0.000445,36.999555
1,1,E09000001,City of London,E09000001,City of London,2011.0,female,1.0,34.000000,0.000398,33.999602
2,2,E09000001,City of London,E09000001,City of London,2011.0,female,2.0,28.000000,0.000372,27.999628
3,3,E09000001,City of London,E09000001,City of London,2011.0,female,3.0,18.000000,0.000359,17.999641
4,4,E09000001,City of London,E09000001,City of London,2011.0,female,4.0,21.000000,0.000367,20.999633
...,...,...,...,...,...,...,...,...,...,...,...
4950395,4950395,E09000033,Westminster,E05013809,Westbourne,2050.0,male,86.0,17.802423,1.413247,16.389175
4950396,4950396,E09000033,Westminster,E05013809,Westbourne,2050.0,male,87.0,13.998027,1.382015,12.616012
4950397,4950397,E09000033,Westminster,E05013809,Westbourne,2050.0,male,88.0,11.476914,1.093119,10.383795
4950398,4950398,E09000033,Westminster,E05013809,Westbourne,2050.0,male,89.0,10.116582,0.858879,9.257702


In [69]:
#select year 2012
household_data[household_data['year'] == 2012]

,Unnamed: 0,gss_code,la_name,gss_code_ward,ward_name,year,sex,age,popn,ce_popn,household_popn
182,182,E09000001,City of London,E09000001,City of London,2012.0,female,0.0,34.000000,0.000445,33.999555
183,183,E09000001,City of London,E09000001,City of London,2012.0,female,1.0,44.000000,0.000398,43.999602
184,184,E09000001,City of London,E09000001,City of London,2012.0,female,2.0,33.000000,0.000372,32.999628
185,185,E09000001,City of London,E09000001,City of London,2012.0,female,3.0,29.000000,0.000359,28.999641
186,186,E09000001,City of London,E09000001,City of London,2012.0,female,4.0,16.000000,0.000367,15.999633
...,...,...,...,...,...,...,...,...,...,...,...
4943479,4943479,E09000033,Westminster,E05013809,Westbourne,2012.0,male,86.0,9.883721,1.413247,8.470474
4943480,4943480,E09000033,Westminster,E05013809,Westbourne,2012.0,male,87.0,10.184049,1.382015,8.802034
4943481,4943481,E09000033,Westminster,E05013809,Westbourne,2012.0,male,88.0,5.854839,1.093119,4.761719
4943482,4943482,E09000033,Westminster,E05013809,Westbourne,2012.0,male,89.0,3.060606,0.858879,2.201727


In [115]:
household_popn_outlier = calculate_zscores_and_find_outliers(household_data, ['household_popn'], handle_inf=False, Geography='ward', z_score_threshold=5, For_population_totals=False)

household_popn_pct_change used Robust Z-Score.


In [116]:
household_popn_outlier['household_popn_pct_change']

,Unnamed: 0,gss_code,la_name,gss_code_ward,ward_name,year,sex,age,popn,ce_popn,household_popn,household_popn_pct_change,robust_z_score
1763578,1763578,E09000012,Hackney,E05009384,Stamford Hill West,2020.0,male,89.0,4.636364,0.506002,4.130361,8408.743189,202056.375424
3329139,3329139,E09000023,Lewisham,E05013719,Deptford,2022.0,male,86.0,2.000000,0.487147,1.512853,3172.661982,76236.185492
1609324,1609324,E09000011,Greenwich,E05014091,West Thamesmead,2013.0,female,80.0,4.978858,0.501654,4.477204,2260.179831,54309.734585
1850381,1850381,E09000013,Hammersmith and Fulham,E05013741,Hammersmith Broadway,2017.0,male,78.0,16.162162,0.490948,15.671214,1920.021651,46135.918114
4413852,4413852,E09000030,Tower Hamlets,E05009325,Lansbury,2022.0,male,79.0,12.000000,0.498368,11.501632,1829.511106,43960.998857
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240437,1240437,E09000009,Ealing,E05013535,Pitshanger,2026.0,male,16.0,98.650537,0.452126,98.198412,0.256362,5.000030
991494,991494,E09000008,Croydon,E05011473,Norbury Park,2018.0,male,49.0,65.621399,0.041626,65.579773,0.256362,5.000028
1328415,1328415,E09000010,Enfield,E05013677,Carterhatch,2029.0,male,88.0,3.024855,0.419416,2.605439,0.256361,5.000026
4376160,4376160,E09000030,Tower Hamlets,E05009320,Bow West,2015.0,male,61.0,44.125356,0.017473,44.107883,0.256361,5.000018
